# Orchestrating Jobs, Model Registration, Continuous Deployment, and Lineage Tracking with Amazon SageMaker

Amazon SageMaker offers Machine Learning application developers and Machine Learning operations engineers the ability to orchestrate SageMaker jobs and author reproducible Machine Learning pipelines, deploy custom-build models for inference in real-time with low latency or offline inferences with Batch Transform, and track lineage of artifacts. You can institute sound operational practices in deploying and monitoring production workflows, deployment of model artifacts, and track artifact lineage through a simple interface, adhering to safety and best-practice paradigmsfor Machine Learning application development.

The SageMaker Workflow service supports a SageMaker Machine Learning Pipeline Domain Specific Language (DSL), which is a declarative Json specification. This DSL defines a Directed Acyclic Graph (DAG) of pipeline parameters and SageMaker job steps. The SageMaker Python Software Developer Kit (SDK) streamlines the generation of the pipeline DSL using constructs that are already familiar to engineers and scientists alike.

The SageMaker Model Registry is where trained models are stored, versioned, and managed. Data Scientists and Machine Learning Engineers can compare model versions, approve models for deployment, and deploy models from different AWS accounts, all from a single Model Registry. SageMaker enables customers to follow the best practices with ML Ops and getting started right. Customers are able to standup a full ML Ops end-to-end system with a single API call.

And the SageMaker Lineage service makes it easy to track all the artifacts created in a SageMaker Machine Learning Pipeline from start to finish.

## SageMaker Pipelines

Amazon SageMaker Pipelines support the following:

* Pipelines - A Directed Acyclic Graph of steps and conditions to orchestrate SageMaker jobs and resource creation.
* Processing Job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* Training Job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* Conditional step execution - Provides conditional execution of branches in a pipeline.
* Registering Models - Creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* Parametrized Pipeline executions - Allows pipeline executions to vary by supplied parameters.
* Transform Job steps - A batch transform to preprocess datasets to remove noise or bias that interferes with training or inference from your dataset, get inferences from large datasets, and run inference when you don't need a persistent endpoint.

## SageMaker Model Registry

Amazon SageMaker Model Registry supports the following:

* Catalog models after the training step - data scientists run tens to thousands of experiments and may select a small set of models as candidates for production.
* Manage model versions - data scientists can register new models which will be automatically versioned in the model registry.
* Compare models - data scientists can run model evaluation steps in Tioga pipeline and generate model metrics (e.g. accuracy metrics and bias metrics) which are recorded in the Model Registry and can be used to compare model versions.
* Approve models - data scientists can mark model versions as “approved” or “rejected”. Alternately, the Tioga pipeline can also automate the model approvals. If there is a deployment pipeline associated with a Model and a live endpoint, then the model version is propagated in production. Currently SageMaker supports a Blue/ Green update, but as part of Yosemite, we are adding support for Canary and Rolling deployments also.
* Deploy models in different AWS accounts - models in the Model Registry support resource sharing across accounts which enables models built in data scientist accounts to be deployed in different pre-production and production accounts.

## SageMaker Lineage

Amazon SageMaker Lineage supports the following:

* Automatically tracks all the artifacts created in a machine learning workflow from start to finish.  Modeled as a directed graph like structure.
* Explore the lineage artifacts with easy to use SDK methods.


## Notebook Overview

This notebook shows how to:

### SageMaker Workflows

* Define a set of Workflow Parameters that can be used to parametrize a Workflow Pipeline
* Define a Processing step that performs cleaning and feature engineering, splitting the input data into train and test data sets
* Define a Training step that trains a model on the pre-processed train data set
* Define a Processing step that evaluates the trained model's performance on the test data set
* Define a Register Model step that creates a model package from the estimator and model artifacts used in training
* Define a Conditional step that measures a condition based on output from prior steps and conditionally executes the Register Model step
* Define and create a Pipeline in a Workflow DAG, with the defined parameters and steps defined
* Start a Pipeline execution and wait for execution to complete
* Download from S3 the model evaluation report for examination
* Start a second Pipeline execution

### SageMaker Model Registry

* Create a SageMaker Project based on the Model Package Group name from the pipeline execution defined before
* Observe CI/CD code pipeline on subsequent successful executions of the pipeline and the registration of a new Model Package version.

### SageMaker Lineage

Amazon SageMaker Lineage supports the following:

* Provide the inputs and outputs of SageMaker job artifacts

# A SageMaker Workflow

The pipeline that we create follows a typical Machine Learning Application pattern of pre-processing, training, evaluation, and model registration:

![A typical ML Application pipeline](img/pipeline-full.png)

### Create SageMaker Clients and Session

First, we create a new SageMaker Session in the `us-east-2` region. We also acquire the role arn for the session.

This role arn should be the execution role arn that you set up in the Prerequisites section of this notebook.

In [1]:
!pip install -q --upgrade pip

In [2]:
!pip install -q sagemaker==2.23.1

In [3]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [5]:
import time
timestamp = str(int(time.time() * 10**7))
print(timestamp)

16092802964020664


# Specify the Raw Inputs S3 Location

In [6]:
raw_input_data_s3_uri = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)
print(raw_input_data_s3_uri)

s3://sagemaker-us-east-1-231218423789/amazon-reviews-pds/tsv/


In [7]:
!aws s3 ls $raw_input_data_s3_uri

2020-12-24 23:31:23 1294879074 amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz
2020-12-18 17:44:16   18997559 amazon_reviews_us_Digital_Software_v1_00.tsv.gz
2020-12-18 17:44:18   27442648 amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz


# Setup Processing Job Hyper-Parameters

In [8]:
max_seq_length=64
train_split_percentage=0.90
validation_split_percentage=0.05
test_split_percentage=0.05
balance_dataset=True
processing_instance_count=1
processing_instance_type='ml.c5.2xlarge'

### Define Parameters to Parametrize Pipeline Execution

We define Workflow Parameters by which we can parametrize our Pipeline and vary the values injected and used in Pipeline executions and schedules without having to modify the Pipeline definition.

The supported parameter types include:

* `ParameterString` - representing a `str` Python type
* `ParameterInteger` - representing an `int` Python type
* `ParameterFloat` - representing a `float` Python type

These parameters support providing a default value, which can be overridden on pipeline execution. The default value specified should be an instance of the type of the parameter.

The parameters defined in this workflow below include:

* `processing_instance_type` - The `ml.*` instance type of the processing job.
* `processing_instance_count` - The instance count of the processing job. For illustrative purposes only: 1 is the only value that makes sense here.
* `training_instance_type` - The `ml.*` instance type of the training job.
* `model_approval_status` - What approval status to register the trained model with for CI/CD purposes. Defaults to "PendingManualApproval". (NOTE: not available in service yet)
* `input_data` - The URL location of the input data

# TODO:  Add this

In [9]:
# from sagemaker.workflow.parameters import (
#     ParameterInteger,
#     ParameterString,
# )

# processing_instance_count = ParameterInteger(
#     name="ProcessingInstanceCount",
#     default_value=1
# )
# processing_instance_type = ParameterString(
#     name="ProcessingInstanceType",
#     default_value="ml.m5.xlarge"
# )
# training_instance_type = ParameterString(
#     name="TrainingInstanceType",
#     default_value="ml.m5.xlarge"
# )
# model_approval_status = ParameterString(
#     name="ModelApprovalStatus",
#     default_value="PendingManualApproval"
# )
# input_data = ParameterString(
#     name="InputData",
#     default_value=input_data_url,
# )

![Define Parameters](img/pipeline-1.png)

We create an instance of an `SKLearnProcessor` processor and we use that in our `ProcessingStep`.

We also specify the `framework_version` we will use throughout.

Note the `processing_instance_type` and `processing_instance_count` parameters that used by the processor instance.

In [10]:
from sagemaker.sklearn.processing import SKLearnProcessor

processor = SKLearnProcessor(framework_version='0.20.0',
                             role=role,
                             instance_type=processing_instance_type,
                             instance_count=processing_instance_count,
                             max_runtime_in_seconds=7200)

In [11]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

processing_inputs=[
        ProcessingInput(
            input_name='raw_input',
            source=raw_input_data_s3_uri,
            destination='/opt/ml/processing/input/data/',
            s3_data_distribution_type='ShardedByS3Key'
        ),
#         ProcessingInput(
#             input_name='code',            
#             source=input_code,
#             destination='/opt/ml/processing/input/code',
#         )
]

#processed_train_data_s3_uri = 's3://{}/{}/processing/output/bert-train'.format(bucket, pipeline_name)        
#processed_validation_data_s3_uri = 's3://{}/{}/processing/output/bert-validation'.format(bucket, pipeline_name)        
#processed_test_data_s3_uri = 's3://{}/{}/processing/output/bert-test'.format(bucket, pipeline_name)

processing_outputs=[
        ProcessingOutput(s3_upload_mode='EndOfJob',
                         output_name='bert-train',
                         source='/opt/ml/processing/output/bert/train',
#                         destination=processed_train_data_s3_uri
                        ),
        ProcessingOutput(s3_upload_mode='EndOfJob',
                         output_name='bert-validation',
                         source='/opt/ml/processing/output/bert/validation',
#                         destination=processed_validation_data_s3_uri
                        ),
        ProcessingOutput(s3_upload_mode='EndOfJob',
                         output_name='bert-test',
                         source='/opt/ml/processing/output/bert/test',
#                         destination=processed_test_data_s3_uri
                        ),
]        

processing_step = ProcessingStep(
    name='Processing', 
    processor=processor,
    inputs=processing_inputs,
    outputs=processing_outputs,
    # experiment_config=experiment_config,
    job_arguments=['--train-split-percentage', str(train_split_percentage),
                   '--validation-split-percentage', str(validation_split_percentage),
                   '--test-split-percentage', str(test_split_percentage),
                   '--max-seq-length', str(max_seq_length),
                   '--balance-dataset', str(balance_dataset)],
    code='preprocess-scikit-text-to-bert.py',    
#    container_entrypoint=['python3', '/opt/ml/processing/input/code/preprocess-scikit-text-to-bert.py'],
)        

print(processing_step)

ProcessingStep(name='Processing', step_type=<StepTypeEnum.PROCESSING: 'Processing'>)


![Define a Processing Step for Feature Engineering](img/pipeline-2.png)

Finally, we use the processor instance to construct a `ProcessingStep`, along with the input and output channels and the code that will be executed when the pipeline invokes pipeline execution. This is very similar to a processor instance's `run` method, for those familiar with the existing Python SDK.

Note the `input_data` parameters passed into `ProcessingStep` as the input data of the step itself. This input data will be used by the processor instance when it is run.

Also, take note the `"train_data"` and `"test_data"` named channels specified in the output configuration for the processing job. Such step `Properties` can be used in subsequent steps and will resolve to their runtime values at execution. In particular, we'll call out this usage when we define our training step.

In [12]:
# !pygmentize src/tf_bert_reviews.py

# Setup Training Hyper-Parameters
Note that `max_seq_length` is re-used from the processing hyper-parameters above

In [13]:
epochs=1
learning_rate=0.00001
epsilon=0.00000001
train_batch_size=128
validation_batch_size=128
test_batch_size=128
train_steps_per_epoch=50
validation_steps=50
test_steps=50
train_instance_count=1
train_instance_type='ml.c5.9xlarge'
train_volume_size=1024
use_xla=True
use_amp=True
freeze_bert_layer=False
enable_sagemaker_debugger=False
enable_checkpointing=False
enable_tensorboard=False
input_mode='File'
run_validation=True
run_test=True
run_sample_predictions=True
deploy_instance_count=1
deploy_instance_type='ml.m5.4xlarge'
# deploy_instance_type='ml.m5.large' # bur

# Setup Metrics To Track Model Performance

In [14]:
metrics_definitions = [
     {'Name': 'train:loss', 'Regex': 'loss: ([0-9\\.]+)'},
     {'Name': 'train:accuracy', 'Regex': 'accuracy: ([0-9\\.]+)'},
     {'Name': 'validation:loss', 'Regex': 'val_loss: ([0-9\\.]+)'},
     {'Name': 'validation:accuracy', 'Regex': 'val_accuracy: ([0-9\\.]+)'},
]

### Define a Training Step to Train a Model

We configure an Estimator and the input dataset. A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later.

We also specify the model path where the models from training will be saved.

Note the `training_instance_type` parameter passed may be also used and passed into other places in the pipeline. In this case, the `training_instance_type` is passed into the estimator.

In [15]:
# from sagemaker.utils import name_from_base
# training_job_name = name_from_base('bert-train')
# print(training_job_name)

In [16]:
# model_path = 's3://{}/{}/model'.format(bucket,training_job_name)
# print(model_path)

In [17]:
from sagemaker.tensorflow import TensorFlow

image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.1.0",
    py_version="py3",
    instance_type="ml.c5.xlarge",
    image_scope="training"
)
print(image_uri)

estimator = TensorFlow(entry_point='tf_bert_reviews.py',
                       source_dir='src',
                       role=role,
#                       output_path=model_path,
#                       base_job_name=training_job_name,
                       instance_count=train_instance_count, # Make sure you have at least this number of input files or the ShardedByS3Key distibution strategy will fail the job due to no data available
                       instance_type=train_instance_type,
                       volume_size=train_volume_size,                       
                       image_uri=image_uri,
#                       py_version='py3',
#                       framework_version='2.1.0',
                       hyperparameters={'epochs': epochs,
                                        'learning_rate': learning_rate,
                                        'epsilon': epsilon,
                                        'train_batch_size': train_batch_size,
                                        'validation_batch_size': validation_batch_size,
                                        'test_batch_size': test_batch_size,                                             
                                        'train_steps_per_epoch': train_steps_per_epoch,
                                        'validation_steps': validation_steps,
                                        'test_steps': test_steps,
                                        'use_xla': use_xla,
                                        'use_amp': use_amp,                                             
                                        'max_seq_length': max_seq_length,
                                        'freeze_bert_layer': freeze_bert_layer,
                                        'enable_sagemaker_debugger': enable_sagemaker_debugger,
                                        'enable_checkpointing': enable_checkpointing,
                                        'enable_tensorboard': enable_tensorboard,                                        
                                        'run_validation': run_validation,
                                        'run_test': run_test,
                                        'run_sample_predictions': run_sample_predictions},
                       input_mode=input_mode,
                       metric_definitions=metrics_definitions,
#                       max_run=7200 # max 2 hours * 60 minutes seconds per hour * 60 seconds per minute
                      )

763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.1.0-cpu-py3


Finally, we use the estimator instance to construct a `TrainingStep` as well as the `Properties` of the prior `ProcessingStep` used as input in the `TrainingStep` inputs and the code that will be executed when the pipeline invokes pipeline execution. This is very similar to an estimator's `fit` method, for those familiar with the existing Python SDK.

In particular, we pass in the `S3Uri` of the `"train_data"` output channel to the `TrainingStep`. We will also use the other `"test_data"` output channel for model evaluation in the pipeline. The `properties` attribute of a Workflow step match the object model of the corresponding response of a describe call. These properties can be referenced as placeholder values and are resolved, or filled in, at runtime. For example, the `ProcessingStep` `properties` attribute matches the object model of the [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response object.

In [18]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep

training_step = TrainingStep(
    name="Train",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs[
                "bert-train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs[
                "bert-validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "test": TrainingInput(
            s3_data=processing_step.properties.ProcessingOutputConfig.Outputs[
                "bert-test"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )        
    },
)

print(training_step)

TrainingStep(name='Train', step_type=<StepTypeEnum.TRAINING: 'Training'>)


![Define a Training Step to Train a Model](img/pipeline-3.png)

### Define a Model Evaluation Step to Evaluate the Trained Model

First, we develop an evaluation script that will be specified in a Processing step that will perform the model evaluation.

The evaluation script `evaluation.py` takes the trained model and the test dataset as input, and produces a JSON file containing classification evaluation metrics, including precision, recall, and F1 score for each label, and accuracy and ROC AUC for the model.

After pipeline execution, we will examine the resulting `evaluation.json` for analysis.

The evaluation script:

* loads in the model
* reads in the test data
* issues a bunch o' predictions against the test data
* builds a classification report, including accuracy and roc
* saves the evaluation report to the evaluation directory

Next, we create an instance of a `ScriptProcessor` processor and we use that in our `ProcessingStep`.

Note the `processing_instance_type` parameter passed into the processor.

In [19]:
# from sagemaker.processing import ScriptProcessor

# script_eval = ScriptProcessor(
#     image_uri=image_uri,
#     command=["python3"],
#     instance_type=processing_instance_type,
#     instance_count=1,
#     base_job_name="script-abalone-eval",
#     sagemaker_session=sess,
#     role=role,
# )

We use the processor instance to construct a `ProcessingStep`, along with the input and output channels and the code that will be executed when the pipeline invokes pipeline execution. This is very similar to a processor instance's `run` method, for those familiar with the existing Python SDK.

In particular, we pass in the `S3ModelArtifacts` from the `TrainingStep`, `step_train` properties as well as the `S3Uri` of the `"test_data"` output channel of the first `ProcessingStep`, `step_process`.

The `TrainingStep` and `ProcessingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) and  [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response objects, respectively.

In [20]:
# from sagemaker.workflow.properties import PropertyFile


# # NOTE:
# # property files cause deserialization failure on listing pipeline executions
# # therefore jsonget and robust conditions won't work
# evaluation_report = PropertyFile(
#     name="EvaluationReport",
#     output_name="evaluation",
#     path="evaluation.json"
# )
# step_eval = ProcessingStep(
#     name="AbaloneEval",
#     processor=script_eval,
#     inputs=[
#         ProcessingInput(
#             source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
#             destination="/opt/ml/processing/model"
#         ),
#         ProcessingInput(
#             source=step_process.properties.ProcessingOutputConfig.Outputs[
#                 "test"
#             ].S3Output.S3Uri,
#             destination="/opt/ml/processing/test"
#         )
#     ],
#     outputs=[
#         ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
#     ],
#     code="evaluation.py",
#     # property_files=[evaluation_report],  # these cause deserialization issues
# )

![Define a Model Evaluation Step to Evaluate the Trained Model](img/pipeline-4.png)

### Define a Register Model Step to Create a Model Package

We use the estimator instance that was used for the training step to construct an instance of `RegisterModel`. The result of executing `RegisterModel` in a pipeline is a Model Package. A Model Package is a reusable model artifacts abstraction that packages all ingredients necessary for inference. Primarily, it consists of an inference specification that defines the inference image to use along with an optional model weights location.

A Model Package Group is a collection of Model Packages. You can create a Model Package Group for a specific ML business problem, and you can keep adding versions/model packages into it. Typically, we expect customers to create a ModelPackageGroup for a SageMaker Workflow Pipeline so that they can keep adding versions/model packages to the group for every Workflow Pipeline run.

The construction of `RegisterModel` is very similar to an estimator instance's `register` method, for those familiar with the existing Python SDK.

In particular, we pass in the `S3ModelArtifacts` from the `TrainingStep`, `step_train` properties. The `TrainingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) response object.

Of note, we provided a specific model package group name which we will use in the Model Registry and CI/CD work later on.

In [21]:
model_package_group_name = f"BERT-Reviews-{timestamp}"

# # NOTE: in the future, the model package group will be created automatically if it doesn't exist
sm.create_model_package_group(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageGroupDescription="BERT-Reviews",
)
print(model_package_group_name)

BERT-Reviews-16092802964020664


In [22]:
inference_image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.1.0",
    py_version="py3",
    instance_type="ml.m5.4xlarge",
    image_scope="inference"
)
print(inference_image_uri)

763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.1.0-cpu


In [23]:
print(training_step.properties.ModelArtifacts.S3ModelArtifacts)

In [24]:
from sagemaker.workflow.step_collections import RegisterModel

# NOTE: model_approval_status is not available as arg in service dsl currently
register_step = RegisterModel(
    name="RegisterModel",
    estimator=estimator,
    image_uri=inference_image_uri, # we have to specify, by default it's using training image
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.m5.4xlarge"],
    transform_instances=["ml.c5.18xlarge"],
    model_package_group_name=model_package_group_name,
    # model_approval_status=model_approval_status,
)

### Define a Condition Step to Check Accuracy and Conditionally Register Model

Finally, we'd like to only register this model if the accuracy of the model, as determined by our evaluation step `step_eval`, exceeded some value. A `ConditionStep` allows for pipelines to support conditional execution in the pipeline DAG based on conditions of step properties. 

Below, we:

* define a `ConditionGreaterThan` on the accuracy value found in the output of the evaluation step, `step_eval`.
* use the condition in the list of conditions in a `ConditionStep`
* pass the `RegisterModel` step collection into the `if_steps` of the `ConditionStep`
* use the `FailStep` in the `else_steps` of the `ConditionStep` to fail the pipeline if the accuracy condition was not met

NOTE: there are a few things that are planned to be implemented in the Workflow service that are currently unavailable to us:

* JsonGet - a function to allow getting json files from S3 and using their values in conditions
* FailStep - a step that terminates the pipeline in failure

In [25]:
# from sagemaker.workflow.conditions import (
#     ConditionEquals,
#     ConditionLessThanOrEqualTo,
# )
# from sagemaker.workflow.condition_step import (
#     ConditionStep,
#     JsonGet,
# )
# from sagemaker.workflow.steps import FailStep


# # NOTE:
# # This is not an ideal condition, but processing jobs have no dynamic properties, as-is
# cond_equals = ConditionEquals(
#     left=step_eval.properties.ProcessingOutputConfig.Outputs["evaluation"].S3Output.S3Uri,
#     right=step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"],
# )

# # NOTE:
# # Ideally, we would use JsonGet to get the mean squared error from the evaluation report
# # This is what a non-trivial condition looks like for a processing job
# #
# # NOTE:
# # jsonpaths only support single characters in the service
# cond_lte = ConditionLessThanOrEqualTo(
#     left=JsonGet(
#         step=step_eval,
#         property_file=evaluation_report,
#         json_path="m"
#     ),
#     right="5.0"
# )

# # NOTE: 
# # we forego the cond_lte and use the cond_equals as jsonget/propertyfiles don't work quite
# # right in service/can't be listed
# #
# # NOTE:
# # FailStep not available in the service yet
# step_cond = ConditionStep(
#     name="AbaloneMSECond",
#     conditions=[cond_equals],  # cond_lte],
#     if_steps=[step_register],
#     else_steps=[],  # [FailStep()]
# )

![Define a Condition Step to Check Accuracy and Conditionally Register Model](img/pipeline-5.png)

### Define a Pipeline of Parameters, Steps, and Conditions

Let's tie it all up into a workflow pipeline so we can execute it, and even schedule it.

A pipeline requires a `name`, `parameters`, and `steps`. Names must be unique within an `(account, region)` pair so we tack on the timestamp to the name.

Note:

* All the parameters used in the definitions must be present.
* Steps passed into the pipeline need not be in the order of execution. The SageMaker Workflow service will resolve the _data dependency_ DAG as steps the execution complete.
* Steps must be unique to either pipeline step list or a single condition step if/else list.

![Define a Pipeline of Parameters, Steps, and Conditions](img/pipeline-6.png)

In [26]:
pipeline_name = 'BERT-pipeline-{}'.format(timestamp)

In [27]:
from sagemaker.workflow.pipeline import Pipeline


# NOTE:
# condition steps have issues in service so we go straight to step_register
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
#        processing_instance_type, 
#        processing_instance_count,
#        training_instance_type,
#        model_approval_status,
#        input_data,
    ],
    steps=[processing_step, training_step, register_step],  # step_cond],
    sagemaker_session=sess,
)

Let's examine the Json of the pipeline definition that meets the SageMaker Workflow Pipeline DSL specification.

By examining the definition, we're also confirming that the pipeline was well-defined, and that the parameters and step properties resolve correctly.

In [29]:
import json
from pprint import pprint

definition = json.loads(pipeline.definition())
pprint(definition)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'Metadata': {},
 'Parameters': [],
 'Steps': [{'Arguments': {'AppSpecification': {'ContainerArguments': ['--train-split-percentage',
                                                                      '0.9',
                                                                      '--validation-split-percentage',
                                                                      '0.05',
                                                                      '--test-split-percentage',
                                                                      '0.05',
                                                                      '--max-seq-length',
                                                                      '64',
                                                                      '--balance-dataset',
                                                                      'True'],
                                               'ContainerEntrypoint': ['python3',
               

### Submit the pipeline to SageMaker and start execution

Let's submit our pipeline definition to the workflow service. The role passed in will be used by the workflow service to create all the jobs defined in the steps.

In [30]:
response = pipeline.create(role_arn=role)

pipeline_arn = response["PipelineArn"]
print(pipeline_arn)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


arn:aws:sagemaker:us-east-1:231218423789:pipeline/bert-pipeline-16092802964020664


We'll start the pipeline, accepting all the default parameters.

Values can also be passed into these pipeline parameters on starting of the pipeline, and will be covered later. 

In [31]:
execution = pipeline.start()
print(execution.arn)

arn:aws:sagemaker:us-east-1:231218423789:pipeline/bert-pipeline-16092802964020664/execution/jvk06bcbvsxx


### Workflow Operations: examining and waiting for pipeline execution

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [32]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:231218423789:pipeline/bert-pipeline-16092802964020664',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:231218423789:pipeline/bert-pipeline-16092802964020664/execution/jvk06bcbvsxx',
 'PipelineExecutionDisplayName': 'execution-1609280383960',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2020, 12, 29, 22, 19, 43, 791000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2020, 12, 29, 22, 19, 43, 791000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:231218423789:user-profile/d-scapt1dubpfo/antje',
  'UserProfileName': 'antje',
  'DomainId': 'd-scapt1dubpfo'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:231218423789:user-profile/d-scapt1dubpfo/antje',
  'UserProfileName': 'antje',
  'DomainId': 'd-scapt1dubpfo'},
 'ResponseMetadata': {'RequestId': '7a57108a-d305-4727-a062-ac94e574a8bb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-reques

In [33]:
execution.list_steps()

[{'StepName': 'Processing',
  'StartTime': datetime.datetime(2020, 12, 29, 22, 19, 44, 293000, tzinfo=tzlocal()),
  'StepStatus': 'Executing',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:231218423789:processing-job/pipelines-jvk06bcbvsxx-processing-v21c4wmlos'}}}]

We can wait for the execution by invoking `wait()` on the execution:

In [34]:
%%time

execution.wait()

CPU times: user 16.9 ms, sys: 0 ns, total: 16.9 ms
Wall time: 106 ms


We can list the execution steps to check out the status and artifacts:

In [35]:
execution.list_steps()

[{'StepName': 'RegisterModel',
  'StartTime': datetime.datetime(2020, 12, 29, 22, 36, 9, 500000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2020, 12, 29, 22, 36, 10, 515000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:231218423789:model-package/bert-reviews-16092802964020664/1'}}},
 {'StepName': 'Train',
  'StartTime': datetime.datetime(2020, 12, 29, 22, 26, 38, 78000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2020, 12, 29, 22, 36, 2, 507000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:231218423789:training-job/pipelines-jvk06bcbvsxx-train-ja5pjvxywe'}}},
 {'StepName': 'Processing',
  'StartTime': datetime.datetime(2020, 12, 29, 22, 19, 44, 293000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2020, 12, 29, 22, 26, 24, 318000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sag

### Examining the evalution

We can examine the resulting model evaluation after the pipeline completes.

We download the resulting `evaluation.json` file from S3 and print the report.

In [36]:
# sagemaker.s3.S3Downloader.read_file("{}/evaluation.json".format(
#     step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
# ))

## Lineage

Let's check out the lineage of the artifacts generated by the pipeline

In [ ]:
# install sagemaker-experiments
# !{sys.executable} -m pip install -q sagemaker-experiments

In [ ]:
#!pip install -q sagemaker-experiments==0.1.25

In [43]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer


viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step['StepName'] == 'Processing':
        processing_job_name=execution_step['Metadata']['ProcessingJob']['Arn'].split('/')[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)


{'StepName': 'Processing', 'StartTime': datetime.datetime(2020, 12, 29, 22, 19, 44, 293000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2020, 12, 29, 22, 26, 24, 318000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:231218423789:processing-job/pipelines-jvk06bcbvsxx-processing-v21c4wmlos'}}}
pipelines-jvk06bcbvsxx-processing-v21c4wmlos


,Name/Source,Direction,Type,Association Type,Lineage Type
0,preprocess-scikit-text-to-bert.py,Input,DataSet,ContributedTo,artifact
1,s3://...2020-12-29-22-19-21-922/output/bert-test,Output,DataSet,Produced,artifact
2,s3://...2-29-22-19-21-922/output/bert-validation,Output,DataSet,Produced,artifact
3,s3://...020-12-29-22-19-21-922/output/bert-train,Output,DataSet,Produced,artifact


{'StepName': 'Train', 'StartTime': datetime.datetime(2020, 12, 29, 22, 26, 38, 78000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2020, 12, 29, 22, 36, 2, 507000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:231218423789:training-job/pipelines-jvk06bcbvsxx-train-ja5pjvxywe'}}}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...2020-12-29-22-19-21-922/output/bert-test,Input,DataSet,ContributedTo,artifact
1,s3://...2-29-22-19-21-922/output/bert-validation,Input,DataSet,ContributedTo,artifact
2,s3://...020-12-29-22-19-21-922/output/bert-train,Input,DataSet,ContributedTo,artifact
3,76310...ws.com/tensorflow-training:2.1.0-cpu-py3,Input,Image,ContributedTo,artifact
4,model.tar.gz,Output,Model,Produced,artifact


{'StepName': 'RegisterModel', 'StartTime': datetime.datetime(2020, 12, 29, 22, 36, 9, 500000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2020, 12, 29, 22, 36, 10, 515000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:231218423789:model-package/bert-reviews-16092802964020664/1'}}}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,bert-reviews-16092802964020664-1-Approved-1609...,Input,Approval,ContributedTo,action
1,model.tar.gz,Input,Model,ContributedTo,artifact
2,76310...onaws.com/tensorflow-inference:2.1.0-cpu,Input,Image,ContributedTo,artifact
3,bert-reviews-16092802964020664-1-PendingManual...,Input,Approval,ContributedTo,action
4,BERT-Reviews-16092802964020664-1609280342-aws-...,Output,ModelGroup,AssociatedWith,context


In [ ]:
# from lineage_visualizer import LineageTableVisualizer

# # NOTE:
# # may take a few times to run as you'll get throttled by lineage service,
# # especially on what appears to be cold starts.
# viz = LineageTableVisualizer(sess)
# for execution_step in execution.list_steps():
#     print(execution_step)
#     viz.show_associations(pipeline_execution_step=execution_step)
#     time.sleep(5)

## The Model Registry and CI/CD


The pipeline that was executed created a Model Package version within the specified Model Package Group. Of particular note, the registration of the model/creation of the Model Package was done so with approval status as `PendingManualApproval`.

Let's check it out:

In [ ]:
for execution_step in execution.list_steps():
    if execution_step["StepName"] == "RegisterModel":
        model_package_arn = execution_step["Metadata"]["RegisterModel"]["Arn"]
        break
print(model_package_arn)

### Update Model Package Approval Status

As noted above, the model has been registered with `"PendingManualApproval"` status. As part of Yosemite, data scientists can register the model with approved/pending manual approval as part of the Tioga workflow. Here we are demonstrating how they can approve the generated model manually. In GA (Nov 2020), we will have UX in SageMaker Studio so that datascients can approve the model, which will inturn trigger the Ci/CD system. For now, here is a way to approve it.

In [ ]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved",
)

print(model_package_update_response)

# Deploy Model

In [ ]:
import time
timestamp = int(time.time())

## Create Model From Model Registry

In [ ]:
model_name = 'bert-model-{}'.format(timestamp)
print("Model name : {}".format(model_name))
primary_container = {
    'ModelPackageName': model_package_arn,
}
create_model_respose = sm.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container
)
print("Model arn : {}".format(create_model_respose["ModelArn"]))

## Create Endpoint Config

In [ ]:
endpoint_config_name = 'bert-model-epc-{}'.format(timestamp)
print(endpoint_config_name)
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m5.4xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

## Create Endpoint

In [ ]:
endpoint_name = 'bert-model-ep-{}'.format(timestamp)
print("EndpointName={}".format(endpoint_name))

create_endpoint_response = sm.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(region, endpoint_name)))


# _Wait Until the Endpoint is Deployed_

In [ ]:
%%time

waiter = sm.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

# Test the Deployed Model

In [ ]:
import json
from sagemaker.tensorflow.model import TensorFlowPredictor

predictor = TensorFlowPredictor(endpoint_name=endpoint_name,
                                sagemaker_session=sess,
                                model_name='saved_model',
                                model_version=0)

# Predict the `star_rating` with Ad Hoc `review_body` Samples

In [ ]:
reviews = ["This is great!"]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)